# Notebook 1 – Data Cleaning, Feature Engineering, & Entity Resolution
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import json
import os
import re
import string
import sys
from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger
from rapidfuzz import fuzz, process

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR
from nomination_predictor.congress_api_utils import \
    enrich_congress_nominees_dataframe

# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

2025-07-12 10:35:29.308 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-12 10:35:29.309 | INFO     | nomination_predictor.config:<module>:127 - Configuration loaded


5

In [ ]:

# ---------------------------------------------------------------------------
# Helper: clean / normalize names
def clean_name(name: str) -> str:
    if pd.isna(name):
        return ""
    name = str(name).upper()
    name = re.sub(r"[\.,]", "", name)          # drop punctuation
    name = re.sub(r"\s+", " ", name).strip()
    return name

def split_name(name: str):
    """
    Very naive splitter: returns first, middle (maybe empty), last
    """
    parts = clean_name(name).split()
    if not parts:
        return "", "", ""
    if len(parts) == 1:
        return parts[0], "", ""
    if len(parts) == 2:
        return parts[0], "", parts[1]
    return parts[0], " ".join(parts[1:-1]), parts[-1]

def normalised_court(text: str) -> str:
    if pd.isna(text):
        return ""
    text = text.upper().replace("UNITED STATES", "").replace("U.S.", "").strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
from nomination_predictor.feature_engineering_helpers import (
    analyze_match_failures, load_and_prepare_dataframes)

# Load and prepare all dataframes
dfs = load_and_prepare_dataframes(RAW_DATA_DIR)
cong_nominees = dfs["cong_nominees"]  # This now has all the derived fields
fjc_judges    = dfs["fjc_judges"]   
fjc_service   = dfs["fjc_service"]
cong_nominations = dfs["cong_nominations"]

print("Loaded:",
      len(fjc_judges), "judges",
      len(fjc_service), "service records",
      len(cong_nominees), "congress nominees",
      len(cong_nominations), "nominations")

2025-07-12 10:35:29 | INFO | load_and_prepare_dataframes - Loaded 4022 judges, 4720 service records, 285 congress nominees, 285 nominations


Loaded: 4022 judges 4720 service records 285 congress nominees 285 nominations


Drop non-judge roles from nominations & nominees list

In [ ]:
# for each row in cong_nominations, if the "nominee_positiontitle" field contains any of the following 
# non-judge-title phrases (case-sensitively matched), then lookup that row's "citation" field and drop the row 
# containing that citation from BOTH the cong_nominations and cong_nominees dataframes, who are expected 
# to each have only had one row with that citation.
non_judicial_titles = ["Attorney","Board","Commission","Director","Marshal","Assistant","Representative","Secretary of","Member of"]

# Make a copy of the dataframes to avoid SettingWithCopyWarning
cong_nominations = cong_nominations.copy()
cong_nominees = cong_nominees.copy()

# Find citations of rows with non-judicial titles
non_judicial_mask = cong_nominations["nominee_positiontitle"].str.contains('|'.join(non_judicial_titles), na=False)
citations_to_drop = cong_nominations.loc[non_judicial_mask, "citation"].unique()

# Log the number of non-judicial nominations being removed
print(f"Found {len(citations_to_drop)} unique citations with non-judicial titles")

# Filter out the non-judicial nominations
original_nominations_count = len(cong_nominations)
original_nominees_count = len(cong_nominees)

cong_nominations = cong_nominations[~cong_nominations["citation"].isin(citations_to_drop)]
cong_nominees = cong_nominees[~cong_nominees["citation"].isin(citations_to_drop)]

# Log the results
print(f"Removed {original_nominations_count - len(cong_nominations)}/{original_nominations_count} non-judicial nominations")
print(f"Removed {original_nominees_count - len(cong_nominees)}/{original_nominees_count} corresponding nominee records")

KeyError: 'nid'

In [ ]:
# --- Clean Congress nominees ------------------------------------------------
cong_nominees["full_name_clean"] = cong_nominees["full_name"].apply(clean_name)
cong_nominees[["first","middle","last"]] = cong_nominees["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

cong_nominees["court_clean"] = cong_nominees["organization"].apply(normalised_court)
cong_nominees["nomination_date"] = pd.to_datetime(cong_nominees["nomination_date"])

# --- Clean FJC judges -------------------------------------------------------
fjc_judges["full_name_clean"] = fjc_judges["name_full"].apply(clean_name)
fjc_judges[["first","middle","last"]] = fjc_judges["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

# We'll need a mapping from nid to service records for date & court validation
fjc_service["court_clean"] = fjc_service["court_name"].apply(normalised_court)
fjc_service["nomination_date"] = pd.to_datetime(fjc_service["nomination_date"], errors="coerce")
fjc_service["commission_date"] = pd.to_datetime(fjc_service["commission_date"], errors="coerce")

In [ ]:

# Block by last name exact match
blocks = {}
for lname, group in fjc_judges.groupby("last"):
    blocks[lname] = group

def candidate_fjc_rows(row):
    return blocks.get(row["last"], pd.DataFrame())

In [ ]:

def best_match(row):
    candidates = candidate_fjc_rows(row)
    if candidates.empty:
        return pd.NA, 0.0
    # Compute combined score: name similarity + court similarity + date proximity
    best_score = 0.0
    best_nid = pd.NA
    for _, cand in candidates.iterrows():
        name_score = fuzz.token_set_ratio(row["full_name_clean"], cand["full_name_clean"])
        # Use service records to find any matching nomination date
        entries = fjc_service[fjc_service["nid"] == cand["nid"]]
        date_score = 0
        court_score = 0
        if not entries.empty:
            # Smallest absolute diff in days
            diffs = (entries["nomination_date"] - row["nomination_date"]).abs().dt.days
            date_score = 100 - diffs.min() if diffs.notna().any() else 0
            # any court string overlap
            if row["court_clean"]:
                if any(row["court_clean"] in c for c in entries["court_clean"]):
                    court_score = 100
                else:
                    court_score = max(fuzz.partial_ratio(row["court_clean"], c) for c in entries["court_clean"])
        total = 0.6*name_score + 0.3*date_score + 0.1*court_score
        if total > best_score:
            best_score, best_nid = total, cand["nid"]
    return best_nid, round(best_score,1)

cong_nominees[["match_nid","match_score"]] = cong_nominees.apply(
    best_match, axis=1, result_type="expand")

In [ ]:

THRESHOLD = 80
matches = cong_nominees[cong_nominees["match_score"] >= THRESHOLD].copy()
print(f"Matched {len(matches)}/{len(cong_nominees)} nominees with score ≥ {THRESHOLD}")
matches.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_matches.csv", index=False)

# Save the cleaned interim datasets for downstream notebooks
cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)

In [ ]:
THRESHOLD = 80
matches = cong_nominees[cong_nominees["match_score"] >= THRESHOLD].copy()
print(f"Matched {len(matches)}/{len(cong_nominees)} nominees with score ≥ {THRESHOLD}")

# Analyze unmatched records to understand why they didn't match
unmatched_df, reason_summary, examples = analyze_match_failures(cong_nominees, THRESHOLD)

# Display summary of failure reasons
print("\nFailure Reason Summary:")
display(reason_summary)

# Display a few examples of each failure type
print("\nExample records for each failure type:")
for reason, example_df in examples.items():
    print(f"\n{reason}:")
    display(example_df)

# Save both matched and unmatched datasets for further analysis
matches.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_matches.csv", index=False)
unmatched_df.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_unmatched.csv", index=False)

# Save the cleaned interim datasets for downstream notebooks
cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)